In [56]:
import json

In [57]:
# Load the OpenAPI JSON file
with open("doc.json", "r") as f:
    data = json.load(f)


In [58]:
def extract_resource_name(path: str) -> str:
    parts = [part for part in path.split('/') if part]
    return parts[-1] if parts else ""

In [59]:
known_collections = ['users', 'stores']

In [60]:
def pb_field_from_schema(name, schema):
    t = schema.get("type")
    nullable = schema.get("nullable", False)
    max_length = schema.get("maxLength")
    minimum = schema.get("minimum")
    maximum = schema.get("maximum")
    _required = ''
    if nullable == False:
        _required = ' , Required: true'
    # Skip ID
    if name == "id":
        return None

    # Handle auto-dates
    if name == "createdAt":
        return f'&core.AutodateField{{Name: "createdAt", OnCreate: true{_required}}}'
    if name == "updatedAt":
        return f'&core.AutodateField{{Name: "updatedAt", OnUpdate: true{_required}}}'

    # Handle relation fields
    if name.endswith("Id"):
        target = f'{name[:-2]}s'  # strip 'Id'
        if target in known_collections:
            return f'&core.RelationField{{Name: "{name}", CollectionId: {target}Col.Id, Required: true}}'

    # Type mappings
    if t == "string":
        if max_length and max_length <= 2048:
            return f'&core.TextField{{Name: "{name}"{_required}}}'
        else:
            return f'&core.TextField{{Name: "{name}"{_required}}}'
    elif t == "boolean":
        return f'&core.BoolField{{Name: "{name}"{_required}}}'
    elif t == "integer" or t == "number":
        min_part = f'Min: types.Pointer[float64]({minimum}),' if minimum is not None else ''
        max_part = f'Max: types.Pointer[float64]({maximum}),' if maximum is not None else ''
        return f'&core.NumberField{{Name: "{name}", {min_part} {max_part}{_required}}}'
    return None

In [ ]:
def extract_relation_fields(properties: dict) -> dict:
    """Return fields matching [name]Id as {field_name: collection_name}."""
    relations = {}
    for field in properties:
        if field.endswith("Id") and field != "id":
            collection = field[:-2]  # remove 'Id' from the end
            relations[field] = f'{collection}s'
    return relations

In [ ]:
# Access the 'paths' object
paths = data.get("paths", {})
# Loop through each path and check for GET methods
i = 2
for path, methods in paths.items():
    if path == '/':
        continue
    if "get" in methods:
        #print(methods)
        print(f"GET path found: {path}")
        # Try to access responses -> 200 -> content -> application/json -> schema -> properties
        try:
            collection = extract_resource_name(path)
            description = methods["get"]["responses"]["200"]["description"]
            tag = methods["get"]["tags"][0]
            if collection in ['me', '{id}', 'featured', 'megamenu']:
                continue
            if collection in known_collections:
                continue
            name = '_'.join(path.split('/')).replace('_api', f'collection{i}')
            known_collections.append(collection)
            print(f"GET path: {path}", collection, tag, description, name)
            #print("Properties:")
            #for prop_name, prop_details in properties.items():
            #    print(f"  - {prop_name}: {prop_details}")
            properties = (
                methods["get"]["responses"]["200"]["content"]["application/json"]["schema"]["properties"]["data"]["items"]["properties"]
            )
            relations = extract_relation_fields(properties)
            _rels = ""
            for rel in relations:
                if relations[rel] in known_collections:
                    _rels += f'{relations[rel]}Col, _ := app.FindCollectionByNameOrId("{relations[rel]}")\n'
            collection_var = collection + "Col"
            code = f'// Migration for collection: {collection} - {tag} - {description}\n'
            code += """package migrations
import (
	"github.com/pocketbase/pocketbase/core"
	"github.com/pocketbase/pocketbase/migrations"
	"github.com/pocketbase/pocketbase/tools/types"
)

func init() {
    migrations.Register(func(app core.App) error {\n"""
            code += f'{_rels}\n'
            code += f'{collection_var}, err := app.FindCollectionByNameOrId("{collection}")\n'
            code += f'if err != nil {{ {collection_var} = core.NewBaseCollection("{collection}") }}\n'
            code += f'{collection_var}.ListRule = types.Pointer("")\n'
            code += f'{collection_var}.ViewRule = types.Pointer("")\n'
            code += f'{collection_var}.Fields.Add(\n'
            field_lines = []
            for prop_name, prop_schema in properties.items():
                line = pb_field_from_schema(prop_name, prop_schema)
                if line:
                    field_lines.append("    " + line + ",")
            code += '\n'.join(field_lines) + '\n)\n'
            code += f'if err := app.Save({collection_var}); err != nil {{ return err }}\n'
            code += 'return nil\n'
            code += """	}, nil)
}\n"""
            print(code)
            with open(f'auto/{name}.go', mode = "w", encoding = "utf-8") as migrations_file:
                migrations_file.write(code)
                migrations_file.close()            
            i += 1
        except Exception as err:
            print(str(err))
            pass
            #print(f"GET path {path} has no '200' response with JSON schema properties.")

GET path found: /api/vendors
GET path: /api/vendors vendors Vendors The list of vendors collection2_vendors
// Migration for collection: vendors - Vendors - The list of vendors
package migrations
import (
	"github.com/pocketbase/pocketbase/core"
	"github.com/pocketbase/pocketbase/migrations"
)

func init() {
    migrations.Register(func(app core.App) error {
vendorsCol, err := app.FindCollectionByNameOrId("vendors")
if err != nil { vendorsCol = core.NewBaseCollection("vendors") }
vendorsCol.ListRule = types.Pointer("")
vendorsCol.ViewRule = types.Pointer("")
vendorsCol.Fields.Add(
    &core.TextField{Name: "status"},
    &core.TextField{Name: "address"},
    &core.TextField{Name: "email"},
    &core.TextField{Name: "phone"},
    &core.TextField{Name: "dialCode"},
    &core.TextField{Name: "name"},
    &core.TextField{Name: "email2"},
    &core.TextField{Name: "banners"},
    &core.TextField{Name: "logo"},
    &core.TextField{Name: "countryName"},
    &core.TextField{Name: "country"},
 